In [136]:
!pip -q install --upgrade beautifulsoup4
from bs4 import BeautifulSoup

In [137]:
import pandas as pd 
import numpy as np
import requests

In [138]:
wiki = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(wiki, 'html5lib')

postcodes_dict = {} 

table = soup.find("table", "wikitable sortable")

for row in table.findAll("tr"):
    cells = row.findAll("td")
    if len(cells) == 3:
        postcode = cells[0].find(text=True)
        borough = cells[1].find(text=True)
        neighborhood = cells[2].find(text=True)
        
        #Ignore cells with a borough that is Not assigned.
        if ( borough != "Not assigned" ):
           
        #More than one neighborhood can exist in one postal code area. hese two rows will be combined into one row with the neighborhoods separated with a comma.
            try:
                neighborhood_old = postcodes_dict[postcode]['neighborhoods']
                  
            except:
                postcodes_dict[postcode] = {}
                postcodes_dict[postcode]['borough'] = borough
                postcodes_dict[postcode]['neighborhoods'] = neighborhood
                #If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
                if ( neighborhood == "Not assigned\n"):
                    neighborhood = borough
                else:
                    neighborhood = neighborhood
            
            else:
                neighborhood_new = neighborhood_old + ', ' + neighborhood
                postcodes_dict[postcode]['neighborhoods'] = neighborhood_new
                
            
                                       
            
          

In [139]:
postcodes_dict

{'M1B': {'borough': 'Scarborough', 'neighborhoods': 'Rouge, Malvern'},
 'M1C': {'borough': 'Scarborough',
  'neighborhoods': 'Highland Creek, Rouge Hill, Port Union'},
 'M1E': {'borough': 'Scarborough',
  'neighborhoods': 'Guildwood\n, Morningside, West Hill'},
 'M1G': {'borough': 'Scarborough', 'neighborhoods': 'Woburn'},
 'M1H': {'borough': 'Scarborough', 'neighborhoods': 'Cedarbrae'},
 'M1J': {'borough': 'Scarborough', 'neighborhoods': 'Scarborough Village'},
 'M1K': {'borough': 'Scarborough',
  'neighborhoods': 'East Birchmount Park\n, Ionview, Kennedy Park'},
 'M1L': {'borough': 'Scarborough',
  'neighborhoods': 'Clairlea, Golden Mile, Oakridge'},
 'M1M': {'borough': 'Scarborough',
  'neighborhoods': 'Cliffcrest, Cliffside, Scarborough Village West\n'},
 'M1N': {'borough': 'Scarborough',
  'neighborhoods': 'Birch Cliff, Cliffside West\n'},
 'M1P': {'borough': 'Scarborough',
  'neighborhoods': 'Dorset Park, Scarborough Town Centre, Wexford Heights'},
 'M1R': {'borough': 'Scarboroug

In [157]:

columns = ['PostalCode', 'Borough', 'Neighborhood']
toronto = pd.DataFrame(columns=columns)

for ind, postcode in enumerate(postcodes_dict):
    borough = postcodes_dict[postcode]['borough']
    neighborhood = postcodes_dict[postcode]['neighborhoods']
    toronto = toronto.append({"PostalCode": postcode, 
                                        "Borough": borough, 
                                        "Neighborhood": neighborhood},
                                        ignore_index=True)

In [158]:
toronto.shape[0]

103

In [159]:
toronto

,PostalCode,Borough,Neighborhood
0,M9M,North York,"Emery, Humberlea"
1,M9A,Etobicoke,Islington Avenue
2,M2H,North York,Hillcrest Village
3,M5M,North York,"Bedford Park, Lawrence Manor East\n"
4,M9N,York,Weston
5,M6B,North York,Glencairn
6,M5S,Downtown Toronto,"Harbord\n, University of Toronto"
7,M1J,Scarborough,Scarborough Village
8,M2N,North York,Willowdale South
9,M5R,Central Toronto,"The Annex, North Midtown\n, Yorkville"


In [164]:
Geocoder = pd.read_csv("https://cocl.us/Geospatial_data")
Geocoder = Geocoder.groupby('Postal Code').first()
Coordinates = toronto.join(Geocoder,on='PostalCode',how="left")
Coordinates

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M9M,North York,"Emery, Humberlea",43.724766,-79.532242
1,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
2,M2H,North York,Hillcrest Village,43.803762,-79.363452
3,M5M,North York,"Bedford Park, Lawrence Manor East\n",43.733283,-79.419750
4,M9N,York,Weston,43.706876,-79.518188
5,M6B,North York,Glencairn,43.709577,-79.445073
6,M5S,Downtown Toronto,"Harbord\n, University of Toronto",43.662696,-79.400049
7,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
8,M2N,North York,Willowdale South,43.770120,-79.408493
9,M5R,Central Toronto,"The Annex, North Midtown\n, Yorkville",43.672710,-79.405678
